# Asynchronous parallel evaluation

This demonstrates an asynchronous evaluation model, or as is known more formally, an asynchronous steady-state evolutionary algorithm (ASEA).  We will demonstrate two approaches.  The first shows a detailed implementation suitable for practitioners that like to have full control of their implementations.  The second shows a more accessible approach using a single, monolithic function that implements a full ASEA.

In [1]:
import sys
from dask.distributed import Client, LocalCluster
import toolz

from leap import core
from leap import ops
from leap import binary_problems
from leap import util
from leap.distributed import asynchronous
from leap.distributed import evaluate

First, let's set up `dask` to run on local pretend "cluster" on your machine.

In [2]:
cluster = LocalCluster()
client = Client(cluster)

Now create an initial random population of five individuals that use a binary representation of four bits for solving the MAX ONES problem.

In [3]:
parents = core.Individual.create_population(5,
                                            initialize=core.create_binary_sequence(4),
                                            decoder=core.IdentityDecoder(), problem=binary_problems.MaxOnes())

To get things started, we send the entire randomly generated initial population to dask to start getting evaluated asynchronously.  We do this by calling `asynchronous.eval_population()`, which returns a distributed dask `as_completed` iterator.  Essentially running `next()` on that iterator will iterate to the next completed individual.

In [4]:
as_completed_iter = asynchronous.eval_population(parents, client=client)

We create a "bag" that will contain the evaluated individuals.  This bag will initially be an empty list.

In [5]:
bag = []

Then we fall into a loop where we insert individuals into a bag with an arbitrary capacity of three.  That means the first three individuals will just be inserted into the bag.  However, the fourth and fifth individual will have to fight it out to be inserted.  We chose the greedy insertion function that means new individuals fight it out with the current *weakest* individual in the bag; there is an alternative function, `insert_into_bag()` that just randomly selects an opponent from the current bag.

To make things more interesting, we will create up to four *new* offspring from the bag. In later, more complex examples, we'll implement a proper births budget to limit the total number of evaluated individuals.

In [6]:
num_offspring = 0

for i, evaluated_future in enumerate(as_completed_iter):
    
    evaluated = evaluated_future.result()
    
    print(i, ', evaluated: ', evaluated.genome, evaluated.fitness)
    
    asynchronous.greedy_insert_into_bag(evaluated, bag, 3)
    
    if num_offspring < 4:
        # Only create offspring if we have the budget for one
        offspring = toolz.pipe(bag,
                               ops.random_selection,
                               ops.clone,
                               ops.mutate_bitflip,
                               ops.pool(size=1))
        print('created offspring:', offspring[0].genome)
        
        # Now asyncrhonously submit to dask
        as_completed_iter.add(client.submit(evaluate.evaluate(context=core.context), offspring[0]))
        
        num_offspring += 1    

0 , evaluated:  [1, 0, 0, 1] 2
created offspring: [1, 0, 1, 1]
1 , evaluated:  [0, 0, 1, 1] 2
created offspring: [1, 0, 1, 0]
2 , evaluated:  [1, 1, 0, 1] 3
created offspring: [0, 0, 1, 0]
3 , evaluated:  [0, 0, 0, 0] 0
created offspring: [1, 0, 0, 1]
4 , evaluated:  [1, 0, 0, 0] 1
5 , evaluated:  [1, 0, 1, 1] 3
6 , evaluated:  [0, 0, 1, 0] 1
7 , evaluated:  [1, 0, 1, 0] 2
8 , evaluated:  [1, 0, 0, 1] 2


Now `bag` should contain the final population of the seven total individuals cooked down to the three best.  Note that there are nine total "evaluated" lines that correspond to the original five randomly generated individuals plus the four new ones we added inside the loop.

In [7]:
[print(i, ind.genome, ind.fitness) for i, ind in enumerate(bag)]

0 [1, 0, 0, 1] 2
1 [0, 0, 1, 1] 2
2 [1, 1, 0, 1] 3
3 [1, 0, 1, 1] 3


[None, None, None, None]